In [ ]:
import os

import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
import dotenv

dotenv.load_dotenv()

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = 'microsoft/Phi-3-mini-128k-instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
)

pipe = HuggingFacePipeline(
    pipeline=pipeline(
        task='text-generation',
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
    )
)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', model_kwargs={'device': 'cuda'})

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_huggingface import ChatHuggingFace

llm = ChatHuggingFace(
    model_id=model_id,
    llm=pipe,
    tokenizer=tokenizer
)

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

vector_store = Chroma.from_documents(documents=splits, embedding=embedding)

In [ ]:
splits

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.prompts import PromptTemplate
from llm.PhiPrompt import PhiPrompt
from langchain_core.prompts import ChatPromptTemplate

promptBuilder = PhiPrompt()

system_prompt = promptBuilder.system_prompt("""You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}""")

user_prompt = promptBuilder.user_prompt("""
        '{input}'
        """)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

print(
    prompt.invoke(
        {
            "context": "recipe for beef wellington",
            "input": "generate 3 recipe."
        }
    )
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
resp = response["answer"]

In [ ]:
resp

In [ ]:
from langchain.chains.history_aware_retriever import  create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do not answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [ ]:
print(contextualize_q_prompt.invoke({
    'input': "what can you do?",
    'chat_history': ['ase', 'nai']
}))

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm, 
    retriever, 
    contextualize_q_prompt
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(pipe, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

In [ ]:
get_session_history("abc124")

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

In [ ]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

retriever_tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)

@tool
def magic_function(input: int) -> int:
    """Applies a magic function to an input."""
    return input + 2

tools = [magic_function, retriever_tool]

In [ ]:
agent = create_tool_calling_agent(llm, tools=tools, prompt=ChatPromptTemplate.from_messages(
    [
        ("system", promptBuilder.system_prompt(
            """You are a helpful assistant.  Chat history: {chat_history}"""
        )),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
))

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
query = "<|user|>What is Task Decomposition?<|end|>"

for s in agent_executor.stream(
        {"input": query,
         "chat_history": chat_history,
        },
):
    print(s['output'])
    print("----")

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

memory = InMemoryChatMessageHistory(
    session_id="abc123"
)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

config = {
    "configurable": {"session_id": "abc123"}
}

In [ ]:
print(
    agent_with_chat_history.invoke(
        {"input": "Hi, I'm polly! What's the output of magic_function of 3?"}, config
    )["output"]
)

print("---")
print(agent_with_chat_history.invoke({"input": "Remember my name?"}, config)["output"])

print("---")
print(
    agent_with_chat_history.invoke({"input": "what was that output again?"}, config)["output"]
)

In [ ]:
tools

In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(llm, tools)